# Data cleaning
We want the class with labels as numerical value and the body with clean text.

This will remove:
* duplicates
* NaN entires
* non english
* url, html

* make it lowercase
* combine title and body

In [2]:
import pandas as pd
import sys
import numpy as np
sys.path.append("../../../scripts_shared/")
from preprocess_text import preprocess_text


In [3]:
# Read CSV into a dataframe
high_priority = pd.read_csv("csv/high/high_priority_no_td.csv", index_col=0)
medium_priority = pd.read_csv("csv/medium/medium_priority_no_td.csv", index_col=0)
low_priority = pd.read_csv("csv/low/low_priority_no_td.csv", index_col=0)

In [4]:
# Number or different labels
high_priority.labels.value_counts().to_frame()[:50]

,count
labels,
bug high priority,10435
high priority,10360
High Priority,7484
enhancement high priority,5486
Priority: High,4751
priority.High,3694
priority.High type.Task,3685
priority.high type.task,3209
priority.High type.Story,2979


In [5]:
#Give each priority a label by number.
# 'Label encoding'. Makes is easier for machine learning models to work with categorical data.
high_priority["labels"] = 0
high_priority["class"] = "high_priority"
medium_priority["labels"] = 1
medium_priority["class"] = "medium_priority"
low_priority["labels"] = 2
low_priority["class"] = "low_priority"
high_priority.head()

,id,type,created_at,repo,repo_url,action,title,labels,body,class
0,1.141754e+10,IssuesEvent,2020-02-03 00:00:44,unitystation/unitystation,https://api.github.com/repos/unitystation/unit...,closed,Client breaking NRE when using edit field on C...,0,### Bug:\r\n\r\nIf you use the edit field of t...,high_priority
1,1.141754e+10,IssuesEvent,2020-02-03 00:01:26,zowe/sample-spring-boot-api-service,https://api.github.com/repos/zowe/sample-sprin...,closed,The SDK provides a separate Java (no-Spring) l...,0,- The commons-spring library is split into:\r\...,high_priority
2,1.141755e+10,IssuesEvent,2020-02-03 00:02:58,openmsupply/mobile,https://api.github.com/repos/openmsupply/mobile,closed,Auto-log out after some time frame,0,## Is your feature request related to a proble...,high_priority
3,1.141755e+10,IssuesEvent,2020-02-03 00:04:18,UltimateCodeMonkeys/CodeMonkeysMVVM,https://api.github.com/repos/UltimateCodeMonke...,opened,Migrate: CodeMonkeys ViewModelNavigationServic...,0,Migrate the Xamarin.Forms navigation service i...,high_priority
4,1.141756e+10,IssuesEvent,2020-02-03 00:08:03,wordpress-mobile/WordPress-Android,https://api.github.com/repos/wordpress-mobile/...,closed,IA Reader filter bottom sheet: manage untitled...,0,In the filter bottom sheet we introduced in th...,high_priority


In [6]:
priority = pd.concat([high_priority, medium_priority, low_priority] , ignore_index = True)

In [7]:
priority[priority["repo"] == "python/mypy"]

,id,type,created_at,repo,repo_url,action,title,labels,body,class


In [8]:
# Remove mypy from the dataset
priority = priority[priority["repo"] != "python/mypy"]
priority

,id,type,created_at,repo,repo_url,action,title,labels,body,class
0,1.141754e+10,IssuesEvent,2020-02-03 00:00:44,unitystation/unitystation,https://api.github.com/repos/unitystation/unit...,closed,Client breaking NRE when using edit field on C...,0,### Bug:\r\n\r\nIf you use the edit field of t...,high_priority
1,1.141754e+10,IssuesEvent,2020-02-03 00:01:26,zowe/sample-spring-boot-api-service,https://api.github.com/repos/zowe/sample-sprin...,closed,The SDK provides a separate Java (no-Spring) l...,0,- The commons-spring library is split into:\r\...,high_priority
2,1.141755e+10,IssuesEvent,2020-02-03 00:02:58,openmsupply/mobile,https://api.github.com/repos/openmsupply/mobile,closed,Auto-log out after some time frame,0,## Is your feature request related to a proble...,high_priority
3,1.141755e+10,IssuesEvent,2020-02-03 00:04:18,UltimateCodeMonkeys/CodeMonkeysMVVM,https://api.github.com/repos/UltimateCodeMonke...,opened,Migrate: CodeMonkeys ViewModelNavigationServic...,0,Migrate the Xamarin.Forms navigation service i...,high_priority
4,1.141756e+10,IssuesEvent,2020-02-03 00:08:03,wordpress-mobile/WordPress-Android,https://api.github.com/repos/wordpress-mobile/...,closed,IA Reader filter bottom sheet: manage untitled...,0,In the filter bottom sheet we introduced in th...,high_priority
...,...,...,...,...,...,...,...,...,...,...
821952,2.060586e+10,IssuesEvent,2022-03-06 23:45:23,bounswe/bounswe2022group1,https://api.github.com/repos/bounswe/bounswe20...,closed,Editing Navigator of Wiki,2,The navigator of the wiki should be edited and...,low_priority
821953,7.334740e+09,IssuesEvent,2018-03-06 00:10:40,hoodedice/notes,https://api.github.com/repos/hoodedice/notes,opened,Check if passwords match,2,JavaScript code (or see if possible without) t...,low_priority
821954,7.334876e+09,IssuesEvent,2018-03-06 00:53:49,zephyrproject-rtos/zephyr,https://api.github.com/repos/zephyrproject-rto...,closed,Add doc to samples/bluetooth/mesh & samples/bl...,2,We should document what exactly the sample is ...,low_priority
821955,7.335047e+09,IssuesEvent,2018-03-06 01:48:07,uwnrg/minotaur-cpp,https://api.github.com/repos/uwnrg/minotaur-cpp,closed,Fix the photo in about dialog xd,2,NaN,low_priority


In [9]:
# Drop duplicates by the content of the body
priority = priority.drop_duplicates(subset=['title'], keep='last')
priority.dropna(inplace=True)
priority.reset_index(inplace=True)
priority.drop(columns=["index"] , inplace= True)
priority["class"].value_counts()

/tmp/ipykernel_228140/2859834588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.dropna(inplace=True)
/tmp/ipykernel_228140/2859834588.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority.drop(columns=["index"] , inplace= True)


class
high_priority      236610
medium_priority    160163
low_priority       143121
Name: count, dtype: int64

In [10]:
# Split the dataframe 
high_priority = priority.loc[priority['class'] == 'high_priority'].copy()
medium_priority = priority.loc[priority['class'] == 'medium_priority'].copy()
low_priority = priority.loc[priority['class'] == 'low_priority'].copy()

In [11]:
high_label_counts = high_priority["class"].value_counts()
medium_label_counts = medium_priority["class"].value_counts()
low_label_counts = low_priority["class"].value_counts()
hp_count = high_label_counts["high_priority"]
print(f"hp count: {hp_count}")
mp_count = medium_label_counts["medium_priority"]
print(f"med count: {mp_count}")
lp_count = low_label_counts["low_priority"]
print(f"low count: {lp_count}")

hp count: 236610
med count: 160163
low count: 143121


In [12]:
high_priority = high_priority.sample(frac=mp_count/hp_count, random_state=42)
high_and_med_priority = pd.concat([medium_priority, high_priority] , ignore_index = True)
high_and_med_priority["class"].value_counts()

class
medium_priority    160163
high_priority      160163
Name: count, dtype: int64

In [13]:
high_and_med_priority["labels"] = 1
low_priority["labels"] = 0
high_and_med_priority["class"] = "high_and_low_priority"
high_and_med_priority["class"].value_counts()

class
high_and_low_priority    320326
Name: count, dtype: int64

In [14]:
high_and_low_label_counts = high_and_med_priority["class"].value_counts()
hm_count = high_and_low_label_counts["high_and_low_priority"]
hm_count

320326

In [15]:

high_and_med_priority = high_and_med_priority.sample(frac=lp_count/hm_count, random_state=42)
all_priority = pd.concat([low_priority, high_and_med_priority] , ignore_index = True)
all_priority["class"].value_counts()

class
low_priority             143121
high_and_low_priority    143121
Name: count, dtype: int64

In [16]:
print(all_priority["title"][0])
print(all_priority["body"][0])

Z-index issue using Classic block
Selecting a link in the classic block and having it active then selecting another link in the left sidebar it looked like this:

![screen shot 2018-05-31 at 18 37 33](https://user-images.githubusercontent.com/5323259/40795551-b05f6796-6502-11e8-8510-65198a4cf7df.png)



In [17]:
# Copy content of body to a new col named text
all_priority["text"] = all_priority["title"] + all_priority["body"]
all_priority.tail()

,id,type,created_at,repo,repo_url,action,title,labels,body,class,text
286237,1.082122e+10,IssuesEvent,2019-11-08 18:09:06,Esri/data-collection-ios,https://api.github.com/repos/Esri/data-collect...,closed,Crash when trying to add image as attachment (...,1,When attempting to add an attachment to a new ...,high_and_low_priority,Crash when trying to add image as attachment (...
286238,1.585694e+10,IssuesEvent,2021-04-08 03:33:27,LcorpOfficial/LCC-Minecraft-Server,https://api.github.com/repos/LcorpOfficial/LCC...,closed,1: Guards,1,write book,high_and_low_priority,1: Guardswrite book
286239,2.622155e+09,IssuesEvent,2015-03-04 00:07:48,byzhang/terrastore,https://api.github.com/repos/byzhang/terrastore,closed,Conditional get/put operations in the Java Client,1,```\nThe Terrastore server supports conditiona...,high_and_low_priority,Conditional get/put operations in the Java Cli...
286240,2.751628e+10,IssuesEvent,2023-03-06 12:12:14,AY2223S2-CS2113-T12-2/tp,https://api.github.com/repos/AY2223S2-CS2113-T...,closed,Navigation Ui,1,Make the Ui keep the states for which menu it ...,high_and_low_priority,Navigation UiMake the Ui keep the states for w...
286241,9.796558e+09,IssuesEvent,2019-06-11 07:56:25,red-hat-storage/ocs-ci,https://api.github.com/repos/red-hat-storage/o...,opened,unittest is failing constantly,1,Please check this failure https://travis-ci.or...,high_and_low_priority,unittest is failing constantly Please check th...


In [18]:
all_priority["text"][0]

'Z-index issue using Classic blockSelecting a link in the classic block and having it active then selecting another link in the left sidebar it looked like this:\r\n\r\n![screen shot 2018-05-31 at 18 37 33](https://user-images.githubusercontent.com/5323259/40795551-b05f6796-6502-11e8-8510-65198a4cf7df.png)\r\n'

In [19]:
# Make a new dataframe with only text, label and class cols.
all_priority_subset = all_priority[["text" , "labels" , "class"]]
all_priority_subset

,text,labels,class
0,Z-index issue using Classic blockSelecting a l...,0,low_priority
1,Indicate the language of a code block**Tell us...,0,low_priority
2,"Use one attribute name for header, title, name...",0,low_priority
3,Document levels of validation in vld_term()As ...,0,low_priority
4,Support puml files in boilerplatessee [this co...,0,low_priority
...,...,...,...
286237,Crash when trying to add image as attachment (...,1,high_and_low_priority
286238,1: Guardswrite book,1,high_and_low_priority
286239,Conditional get/put operations in the Java Cli...,1,high_and_low_priority
286240,Navigation UiMake the Ui keep the states for w...,1,high_and_low_priority


In [20]:
# Convert to string
all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)

/tmp/ipykernel_228140/3106036705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_str"] = all_priority_subset['text'].astype(str)


In [21]:
all_priority_subset

,text,labels,class,text_str
0,Z-index issue using Classic blockSelecting a l...,0,low_priority,Z-index issue using Classic blockSelecting a l...
1,Indicate the language of a code block**Tell us...,0,low_priority,Indicate the language of a code block**Tell us...
2,"Use one attribute name for header, title, name...",0,low_priority,"Use one attribute name for header, title, name..."
3,Document levels of validation in vld_term()As ...,0,low_priority,Document levels of validation in vld_term()As ...
4,Support puml files in boilerplatessee [this co...,0,low_priority,Support puml files in boilerplatessee [this co...
...,...,...,...,...
286237,Crash when trying to add image as attachment (...,1,high_and_low_priority,Crash when trying to add image as attachment (...
286238,1: Guardswrite book,1,high_and_low_priority,1: Guardswrite book
286239,Conditional get/put operations in the Java Cli...,1,high_and_low_priority,Conditional get/put operations in the Java Cli...
286240,Navigation UiMake the Ui keep the states for w...,1,high_and_low_priority,Navigation UiMake the Ui keep the states for w...


In [22]:
# Clean the data.
all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)

/tmp/ipykernel_228140/63294665.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_priority_subset["text_clean"] = all_priority_subset["text_str"].map(preprocess_text)


In [23]:
# Make a subset with text_clean and label
priority_label_text = all_priority_subset[["text_clean" , "labels", "class"]]
priority_label_text

,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
286237,crash when trying to add image as attachment i...,1,high_and_low_priority
286238,1 guardswrite book,1,high_and_low_priority
286239,conditional getput operations in the java clie...,1,high_and_low_priority
286240,navigation uimake the ui keep the states for w...,1,high_and_low_priority


In [24]:
# Need to dropna here since cleaning function returns NaN for not english text.
priority_label_text.dropna(inplace=True)
priority_label_text.reset_index(inplace=True)
priority_label_text.drop(columns=["index"] , inplace= True)

priority_label_text

/tmp/ipykernel_228140/239147930.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.dropna(inplace=True)
/tmp/ipykernel_228140/239147930.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priority_label_text.drop(columns=["index"] , inplace= True)


,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
267979,crash when trying to add image as attachment i...,1,high_and_low_priority
267980,1 guardswrite book,1,high_and_low_priority
267981,conditional getput operations in the java clie...,1,high_and_low_priority
267982,navigation uimake the ui keep the states for w...,1,high_and_low_priority


In [25]:
# Splitting the DataFrame
test_df = priority_label_text.sample(frac=0.05, random_state=1)  # Select 5% of the data
priority_df = priority_label_text.drop(test_df.index) 
priority_df.reset_index(drop=True, inplace=True)
priority_df

,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
254580,crash when trying to add image as attachment i...,1,high_and_low_priority
254581,1 guardswrite book,1,high_and_low_priority
254582,conditional getput operations in the java clie...,1,high_and_low_priority
254583,navigation uimake the ui keep the states for w...,1,high_and_low_priority


In [26]:
test_file_name = f"csv/clean_test_low_vs_high_and_med_priority.csv"
test_df.to_csv(test_file_name, index=False)

In [27]:
# Clean dataset with clean text and labels.
# 0 = high priority, 1 = not high priority
file_name = f"csv/clean_low_vs_high_and_med_priority.csv"
priority_df.to_csv(file_name, index=False)

In [28]:
pri = pd.read_csv(file_name)
pri

,text_clean,labels,class
0,zindex issue using classic blockselecting a li...,0,low_priority
1,indicate the language of a code blocktell us a...,0,low_priority
2,use one attribute name for header title namema...,0,low_priority
3,document levels of validation in vldtermas thi...,0,low_priority
4,support puml files in boilerplatessee this com...,0,low_priority
...,...,...,...
254580,crash when trying to add image as attachment i...,1,high_and_low_priority
254581,1 guardswrite book,1,high_and_low_priority
254582,conditional getput operations in the java clie...,1,high_and_low_priority
254583,navigation uimake the ui keep the states for w...,1,high_and_low_priority


In [29]:
test_df = pd.read_csv(test_file_name)
test_df

,text_clean,labels,class
0,social meta module add support for twitter app...,0,low_priority
1,probecli remove the sendcrashreports option fr...,0,low_priority
2,add steps next to points on rankings pageso on...,1,high_and_low_priority
3,service change white list to support list in s...,1,high_and_low_priority
4,support words selection in text nodes with a s...,0,low_priority
...,...,...,...
13394,missing translation strings in offline editors...,1,high_and_low_priority
13395,coverity cid 219641 outofbounds access in test...,0,low_priority
13396,last blog post on main menu is outdated descri...,0,low_priority
13397,lifecyclecostusepriceescalation should impact ...,1,high_and_low_priority
